In [2]:
import pandas as pd

quick = True

if quick:
    start_year, end_year, chk, sampled, filename = [2005, 2021, False, True, 'df-light.pkl']
else:
    start_year, end_year, chk, sampled, filename = [2005, 2021, True, False, 'df-full.pkl']

from sklearn.model_selection import train_test_split

df = pd.read_pickle(f'./{filename}')
data = df.iloc[:, 1:]
target = df['grav']

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=222)

In [6]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from category_encoders import TargetEncoder, OneHotEncoder
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.metrics import classification_report_imbalanced
import time

start_time = time.time()

cols_target_encoded = ['dep', 'age']
cols_onehot_encoded = X_train.columns.drop(cols_target_encoded)
scaler = StandardScaler()

encoder_target = TargetEncoder(cols=cols_target_encoded)
encoder_onehot = OneHotEncoder(cols=cols_onehot_encoded)
# sampler        = SMOTE(random_state=42)
sampler        = RandomUnderSampler()
# sampler        = RandomOverSampler()
model          = GradientBoostingClassifier()


X_train_te = encoder_target.fit_transform(X_train, y_train)
X_test_te  = encoder_target.transform(X_test)

X_train_oh = encoder_onehot.fit_transform(X_train_te, y_train)
X_test_oh  = encoder_onehot.transform(X_test_te)

X_train_sc = scaler.fit_transform(X_train_oh)
X_test_sc  = scaler.transform(X_test_oh)

X_train_rs, y_train_rs = sampler.fit_resample(X_train_sc, y_train)
X_test_rs, y_test_rs   = sampler.fit_resample(X_test_sc, y_test)

params = {
    'n_estimators' : [100],
    'criterion' : ('friedman_mse', 'squared_error'),
    # 'max_depth' : (None, 10, 50),
    # 'min_samples_split' : (2,4,6)
}

grid = GridSearchCV(estimator=model, param_grid=params, cv = 3, n_jobs=-1)
grid.fit(X_train_rs, y_train_rs)

print('Best score  : ', grid.best_score_)
print('Best params : ', grid.best_params_)

y_pred = grid.predict(X_test_rs)
print(classification_report_imbalanced(y_test_rs, y_pred))

print("--- performed in %s seconds ---" % (time.time() - start_time))

Best score  :  0.7423486527765922
Best params :  {'criterion': 'squared_error', 'n_estimators': 100}
                   pre       rec       spe        f1       geo       iba       sup

          0       0.76      0.71      0.77      0.73      0.74      0.55      4469
          1       0.73      0.77      0.71      0.75      0.74      0.55      4469

avg / total       0.74      0.74      0.74      0.74      0.74      0.55      8938

--- performed in 145.93663597106934 seconds ---
